In [119]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [120]:
farms = pd.read_csv("../data-mining/farms.csv")
emissions = pd.read_csv("registre-francais-des-emission-polluantes-emissions.csv")

In [121]:
emissions = emissions[emissions.Milieu == "Air"]

In [122]:
farms.name = farms.name.str.upper()
emissions["Nom Etablissement"] = emissions["Nom Etablissement"].str.upper()

In [123]:
merged = farms.merge(emissions, left_on="name", right_on="Nom Etablissement")

In [124]:
def departement_of(code_postal: int) -> int:
    departement = int(str(code_postal)[:2])

    # dom_tom departement code is 3 digits-long instead of 2
    dom_tom: bool = departement == 97
    if dom_tom:
        departement = int(str(code_postal)[:3])

    return departement

#emissions = emissions.dropna(subset="Code Postal")
#emissions["Code Postal"] = emissions["Code Postal"].astype(int)

#emissions["Departement"] = emissions["Code Postal"].apply(departement_of)

merged = merged.dropna(subset="Code Postal")
merged["Code Postal"] = merged["Code Postal"].astype(int)

merged["Departement"] = merged["Code Postal"].apply(departement_of)

In [125]:
emissions_dpt = pd.DataFrame({
    'nh3_sum_over_years' : merged.query("Polluant == 'Ammoniac (NH3)'")\
                                    .groupby(["Nom Etablissement", "Departement"])["Quantité"].sum()
            }).reset_index()

In [126]:
emissions_dpt

,Nom Etablissement,Departement,nh3_sum_over_years
0,AUGUIN DOMINIQUE,85,10400.0
1,AVI-BIZ (EARL),49,25200.0
2,AVIVRESNE (GAEC),49,48900.0
3,AXIOM,86,65300.0
4,BELIN MARCEL,32,24100.0
...,...,...,...
225,SOCIETE AUREGAN MEGRET,22,116500.0
226,TRICOIRE LAETITIA,85,15200.0
227,URVOY JOELLE,22,11900.0
228,VALANGLAISE (GAEC),49,22200.0


In [127]:
result = pd.DataFrame({
    'num_fermes_emission_massive' :emissions_dpt.groupby(["Departement"])["nh3_sum_over_years"].count()
            }).reset_index()

In [128]:
result

,Departement,num_fermes_emission_massive
0,10,1
1,11,1
2,14,1
3,16,2
4,18,1
5,22,42
6,25,1
7,26,9
8,29,41
9,32,2


In [129]:
result.to_json("5.json", orient="records")